In [1]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import csv

DATA = os.path.abspath("../data")
INPUT_PATH = os.path.join(DATA, "generated")

def read_smiles(file_name):
    with open(file_name, "r") as f:
        reader = csv.reader(f)
        smiles = []
        for r in reader:
            smiles += [r[0]]
    return smiles

def write_smiles(smiles_list, file_name):
    with open(file_name, "w") as f:
        writer = csv.writer(f)
        for smi in smiles_list:
            writer.writerow([smi])

In [2]:
import collections
import urllib
import json

from smallworld_api import SmallWorld

import warnings
warnings.filterwarnings("ignore")


def get_available_maps():
    url = "https://sw.docking.org/search/maps"
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data


def get_maps():
    data = get_available_maps()
    labels = ["REAL", "WuXi", "MCule", "Zinc"]
    found_maps = collections.defaultdict(list)
    for k,v in data.items():
        for l in labels:
            l_ = l.lower()
            k_ = k.lower()
            if l_ in k_:
                found_maps[l] += [k]
    found_maps_ = {}
    for k,v in found_maps.items():
        if len(v) == 1:
            v_ = v[0]
            if data[v_]["enabled"] and data[v_]["status"] == "Available":
                found_maps_[k] = v_
        else:
            v_sel = None
            w_sel = None
            for v_ in v:
                if not data[v_]["enabled"] or data[v_]["status"] != "Available":
                    continue
                w_ = data[v_]["numEntries"]
                if v_sel is None:
                    v_sel = v_
                    w_sel = w_
                else:
                    if w_ > w_sel:
                        v_sel = v_
                        w_sel = w_
            if v_sel is not None:
                found_maps_[k] = v_sel
    result = []
    for l in labels:
        if l in found_maps_:
            result += [(l, found_maps_[l])]
    return result

In [10]:
from tqdm import tqdm
import time
import random


class SmallWorldSampler(object):
    
    def __init__(self, dist=10, length=100):
        self.maps = get_maps()
        self.sw = SmallWorld()
        self.dist = dist
        self.length = length
        self.seconds_per_query = 3
        
    def _sample(self, smiles, time_budget_sec):
        t0 = time.time()
        sampled_smiles = []
        for m in self.maps:
            try:
                db_name = m[1]
                results : pd.DataFrame = self.sw.search(smiles, dist=self.dist, db=db_name, length=self.length)
            except:
                print(smiles, m, "did not work...")
                results = None
            if results is not None:
                sampled_smiles += list(results["smiles"])
            t1 = time.time()
            if (t1-t0) > time_budget_sec:
                break
            t0 = time.time()
        return sampled_smiles
            
    def sample(self, smiles_list, time_budget_sec=600):
        time_budget_sec_per_query = int(time_budget_sec/(self.seconds_per_query*len(smiles_list)))+1
        sampled_smiles = []
        for smi in tqdm(smiles_list):
            sampled_smiles += self._sample(smi, time_budget_sec_per_query)
        sampled_smiles = list(set(sampled_smiles))
        random.shuffle(sampled_smiles)
        return sampled_smiles

smp = SmallWorldSampler()

In [11]:
smiles = read_smiles(os.path.join(INPUT_PATH, "known_hits.csv"))

sampled_smiles = smp.sample(smiles, 600)
write_smiles(sampled_smiles, os.path.join(INPUT_PATH, "known_hits_lib_aug.csv"))

 12%|██████████▉                                                                            | 2/16 [01:02<07:25, 31.79s/it]

O=C(C1CCN(C)CC1)NC2=C(C3=CC=C(C=CC=C4)C4=C3)C=CC=C2 ('MCule', 'MculeUltimate-20Q2.smi.anon') did not work...


 56%|████████████████████████████████████████████████▉                                      | 9/16 [04:37<03:28, 29.77s/it]

CN(CC1)CCN1C2=CC=C(C3=CC=C(OC4=CC=CC=C4)C=C3)C=C2 ('Zinc', 'all-zinc.smi.anon') did not work...


 69%|███████████████████████████████████████████████████████████▏                          | 11/16 [05:27<02:11, 26.35s/it]

...

 88%|███████████████████████████████████████████████████████████████████████████▎          | 14/16 [06:39<00:51, 25.79s/it]

CN(CC1)CCN1C2=CC=C(NC(CN3CCC(C=CC=C4)=C4C3)=O)C=C2 ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


 94%|████████████████████████████████████████████████████████████████████████████████▋     | 15/16 [07:12<00:28, 28.10s/it]

CN(CC1)CCN1C2=CC=C(NC(CN3CCC(C=CC=C4)=C4C3)=O)C=C2 ('MCule', 'MculeUltimate-20Q2.smi.anon') did not work...


100%|██████████████████████████████████████████████████████████████████████████████████████| 16/16 [07:37<00:00, 28.60s/it]


In [12]:
smiles = read_smiles(os.path.join(INPUT_PATH, "pocketvec_mw250_hits.csv"))

sampled_smiles = smp.sample(smiles, 600)
write_smiles(sampled_smiles, os.path.join(INPUT_PATH, "pocketvec_mw250_hits_lib_aug.csv"))

  7%|█████▊                                                                                 | 1/15 [00:20<04:47, 20.54s/it]

O=C(O)C(=O)/C=C/[C@@H](F)C(=O)c1ccccc1F ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


 20%|█████████████████▍                                                                     | 3/15 [01:14<05:17, 26.50s/it]

Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)[C@@H](O)[C@H]1O ('MCule', 'MculeUltimate-20Q2.smi.anon') did not work...


 27%|███████████████████████▏                                                               | 4/15 [01:53<05:47, 31.60s/it]

...

CC(=O)N[C@@H](Cc1ccc2ccccc2c1)C(=O)O ('MCule', 'MculeUltimate-20Q2.smi.anon') did not work...


 80%|████████████████████████████████████████████████████████████████████▊                 | 12/15 [07:16<02:09, 43.02s/it]

Cc1ccc(S(=O)(=O)N2CCSC[C@H]2C(=O)N[C@@H](CC(C)C)C(=O)O)cc1 ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


100%|██████████████████████████████████████████████████████████████████████████████████████| 15/15 [08:49<00:00, 35.31s/it]

O=c1[nH]c2cc(C(F)(F)F)c(N3CCOCC3)cc2n(CP(=O)(O)O)c1=O ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


In [13]:
smiles = read_smiles(os.path.join(INPUT_PATH, "pocketvec_hits.csv"))

sampled_smiles = smp.sample(smiles, 600)
write_smiles(sampled_smiles, os.path.join(INPUT_PATH, "pocketvec_hits_lib_aug.csv"))

  6%|█████▍                                                                                 | 5/80 [01:43<21:43, 17.37s/it]

Nc1c(O)cccc1C(=O)O ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


  8%|██████▌                                                                                | 6/80 [01:52<18:14, 14.78s/it]

O=C(O)C(=O)Cc1c[nH]c2ccccc12 ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


 12%|██████████▊                                                                           | 10/80 [02:53<15:51, 13.59s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████████▊   | 77/80 [19:46<00:44, 14.86s/it]

CCO[C@@H]1C=CC=C(C(=O)O)[C@H]1N ('WuXi', 'WuXi-20Q4.smi.anon') did not work...


 99%|████████████████████████████████████████████████████████████████████████████████████▉ | 79/80 [20:16<00:14, 14.65s/it]

O=C(O)c1ccc(Br)cc1 ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


100%|██████████████████████████████████████████████████████████████████████████████████████| 80/80 [20:56<00:00, 15.70s/it]


In [14]:
smiles = read_smiles(os.path.join(INPUT_PATH, "docking_top100_hits.csv"))

sampled_smiles = smp.sample(smiles, 600)
write_smiles(sampled_smiles, os.path.join(INPUT_PATH, "docking_top100_hits_lib_aug.csv"))

  5%|████▎                                                                                 | 5/100 [01:27<19:57, 12.61s/it]

Cc1ccc(S(=O)(=O)Nc2nc3c(C)cc(NC(=O)c4cc5ccccc5[nH]4)cc3s2)cc1 ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


  6%|█████▏                                                                                | 6/100 [01:36<18:04, 11.53s/it]

Cc1ccc(S(=O)(=O)Nc2nc3c(C)cc(NC(=O)c4cc5ccccc5[nH]4)cc3s2)cc1 ('WuXi', 'WuXi-20Q4.smi.anon') did not work...


 10%|████████▌                                                                            | 10/100 [02:05<12:42,  8.47s/it]

...

 95%|████████████████████████████████████████████████████████████████████████████████▊    | 95/100 [13:50<00:34,  6.99s/it]

O=C(c1ccccc1)N1CCC(c2nc(C3CCOCC3)no2)C1 ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


 97%|██████████████████████████████████████████████████████████████████████████████████▍  | 97/100 [14:07<00:23,  7.86s/it]

N#CC1=C2N=C(N)c3c(sc4c3CCCCC4)N2C2=C(C(=O)CCC2)C1c1cccs1 ('REAL', 'REAL-Database-22Q1.smi.anon') did not work...


100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [14:36<00:00,  8.76s/it]
